In [1]:
!pip install -U git+https://github.com/huggingface/transformers -q

In [2]:
!wget https://d8it4huxumps7.cloudfront.net/files/66e31d6ee96cd_student_resource_3.zip
!unzip 66e31d6ee96cd_student_resource_3.zip
!rm 66e31d6ee96cd_student_resource_3.zip

--2024-09-14 08:38:41--  https://d8it4huxumps7.cloudfront.net/files/66e31d6ee96cd_student_resource_3.zip
Resolving d8it4huxumps7.cloudfront.net (d8it4huxumps7.cloudfront.net)... 18.65.229.18, 18.65.229.59, 18.65.229.45, ...
Connecting to d8it4huxumps7.cloudfront.net (d8it4huxumps7.cloudfront.net)|18.65.229.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5184192 (4.9M) [application/zip]
Saving to: '66e31d6ee96cd_student_resource_3.zip'

66e31d6ee96cd_stude 100%[===================>]   4.94M  --.-KB/s    in 0.07s   

2024-09-14 08:38:41 (75.3 MB/s) - '66e31d6ee96cd_student_resource_3.zip' saved [5184192/5184192]

Archive:  66e31d6ee96cd_student_resource_3.zip
   creating: student_resource 3/
  inflating: __MACOSX/._student_resource 3  
  inflating: student_resource 3/sample_code.py  
  inflating: __MACOSX/student_resource 3/._sample_code.py  
  inflating: student_resource 3/.DS_Store  
  inflating: __MACOSX/student_resource 3/._.DS_Store  
   creating: studen

In [3]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
import pandas as pd
from torchvision import transforms
from PIL import Image
import requests
from io import BytesIO
import os
from tqdm import tqdm


In [4]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

english = {}


for k in entity_unit_map.keys():
    english[k] = k
    
english["item_weight"] = "weight"
english["item_volume"] = "volume"


# Setup

In [5]:
import pandas as pd
df = pd.read_csv("/kaggle/working/student_resource 3/dataset/train.csv")

In [6]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

<h1> Download images parallely </h1>


In [39]:
subset = df[:20]

print(subset["image_link"])

0     https://m.media-amazon.com/images/I/61I9XdN6OF...
1     https://m.media-amazon.com/images/I/71gSRbyXmo...
2     https://m.media-amazon.com/images/I/61BZ4zrjZX...
3     https://m.media-amazon.com/images/I/612mrlqiI4...
4     https://m.media-amazon.com/images/I/617Tl40LOX...
5     https://m.media-amazon.com/images/I/61QsBSE7jg...
6     https://m.media-amazon.com/images/I/81xsq6vf2q...
7     https://m.media-amazon.com/images/I/71DiLRHeZd...
8     https://m.media-amazon.com/images/I/91Cma3Rzse...
9     https://m.media-amazon.com/images/I/71jBLhmTNl...
10    https://m.media-amazon.com/images/I/81N73b5khV...
11    https://m.media-amazon.com/images/I/61oMj2iXOu...
12    https://m.media-amazon.com/images/I/91LPf6OjV9...
13    https://m.media-amazon.com/images/I/81fOxWWWKY...
14    https://m.media-amazon.com/images/I/81dzao1Ob4...
15    https://m.media-amazon.com/images/I/91-iahVGED...
16    https://m.media-amazon.com/images/I/81S2+GnYpT...
17    https://m.media-amazon.com/images/I/81e2Yt

In [8]:
!pip install constants

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of constants to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
  Created wheel for constants: filename=constants-0.6.0-py3-none-any.whl size=5458 sha256=edc823dc36abd167d556f7ecd43504f346c4629156ff664e38c4df01c89c3dba
  Stored in directory: /root/.cache/pip/wheels/5b/96/3c/386c2342a8a1bdd317f2f250bd076c13938c6f598c4a40ec14
Successfully built constants


In [40]:
import re
import constants
import os
import requests
import pandas as pd
import multiprocessing
import time
from time import time as timer
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import requests
import urllib
from PIL import Image

def create_placeholder_image(image_save_path):
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        return

def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)
    print(image_save_path)

    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except:
            time.sleep(delay)
    
    create_placeholder_image(image_save_path) #Create a black placeholder image for invalid links/images

def download_images(image_links, download_folder, allow_multiprocessing=True):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(
            download_image, save_folder=download_folder, retries=3, delay=3)

        with multiprocessing.Pool(64) as pool:
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=3, delay=3)
        

In [41]:
import os
os.makedirs("downloaded_images", exist_ok=True)


save_dir = "downloaded_images"
image_urls = subset['image_link'].tolist()

download_images(image_urls, save_dir, allow_multiprocessing=True)

downloaded_images/61I9XdN6OFL.jpgdownloaded_images/612mrlqiI4L.jpgdownloaded_images/71gSRbyXmoL.jpgdownloaded_images/61QsBSE7jgL.jpgdownloaded_images/61BZ4zrjZXL.jpg



  0%|          | 0/20 [00:00<?, ?it/s]


downloaded_images/81xsq6vf2qL.jpg
downloaded_images/617Tl40LOXL.jpgdownloaded_images/71jBLhmTNlL.jpg

downloaded_images/81S2+GnYpTL.jpgdownloaded_images/91Cma3RzseL.jpgdownloaded_images/81RNsNEM1EL.jpgdownloaded_images/91-iahVGEDL.jpgdownloaded_images/81dzao1Ob4L.jpgdownloaded_images/81N73b5khVL.jpgdownloaded_images/91prZeizZnL.jpgdownloaded_images/91LPf6OjV9L.jpg
downloaded_images/71DiLRHeZdL.jpgdownloaded_images/81e2YtCOKvL.jpg

downloaded_images/81fOxWWWKYL.jpg

downloaded_images/61oMj2iXOuL.jpg










100%|██████████| 20/20 [00:00<00:00, 68.23it/s]


# Inference

In [42]:
transform = transforms.Compose([
    transforms.Resize((600, 600)),
    transforms.ToTensor(),
])


In [43]:
from PIL import Image
def url_to_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    resized = trans(img)
    return resized

# Replace the 'image_link' column with PIL.Image objects


In [44]:
def get_message(entity_name):
    message = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                    },
                    {"type": "text", "text": f"What is the {english[entity_name]} of the item in the image"},
                ],
            }
        ]
    return message

In [45]:
from typing import List
def forward(entity_names, images):
    """
    entity_name -> It is a list of entity names in the batch
    images -> List of images
    """
    
    messages = [ get_message(entity)  for entity in entity_names]
    
    texts = [
        processor.apply_chat_template(msg, add_generation_prompt=True) 
        for msg in messages ]

    
    #text = processor.apply_chat_template(messages, add_generation_prompt=True)

    
    inputs = processor(
        text=texts, 
        images=images, 
        padding=True, 
        return_tensors="pt"
    )

    inputs = inputs.to("cuda")
    
    
    output_ids = model.generate(**inputs, max_new_tokens=32)
    
    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(inputs.input_ids, output_ids)
    ]
    del inputs
    output_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )

    del generated_ids
    del output_ids

    
    
    
    return output_text

In [46]:
def get_local_filename(image_link):
    # Extract just the filename from the URL
    return os.path.basename(image_link)

In [47]:
imgs = []
enames = []
values = []

In [48]:

for ix, row in subset.iterrows():
    image_link = row["image_link"]
    local_filename = get_local_filename(image_link)
    img_path = os.path.join("downloaded_images", local_filename)
    
    if os.path.exists(img_path):
        # Open and transform the image
        with Image.open(img_path).convert('RGB') as img:
            img_tensor = transform(img)
        imgs.append(img_tensor)
        
        ename = row["entity_name"]
        enames.append(ename)
        
        value = row["entity_value"]  # Assuming entity_value is in the same row
        values.append(value)
    else:
        print(f"Warning: File not found: {img_path}")

In [50]:
import time 
import torch

images_tensor = torch.stack(imgs)

start = time.time()
output = forward(enames, imgs)
end = time.time()

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


OutOfMemoryError: CUDA out of memory. Tried to allocate 37.10 GiB. GPU 0 has a total capacity of 15.89 GiB of which 6.56 GiB is free. Process 5388 has 9.33 GiB memory in use. Of the allocated memory 8.63 GiB is allocated by PyTorch, and 427.43 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
execution_time = end - start  # Calculate the difference
print(f"Execution time: {execution_time} seconds")


In [ ]:
import re

# Dictionary to map unit abbreviations to full forms
unit_mapping = {
    'g': 'gram',
    'kg': 'kilogram',
    'mg': 'milligram',
    'ug': 'microgram',
    'lb': 'pound',
    'oz': 'ounce',
    'ton': 'ton',
    'mm': 'millimeter',
    'cm': 'centimeter',
    'm': 'meter',
    'ft': 'foot',
    'in': 'inch',
    'yd': 'yard'
}

# Function to convert the sentence
def extract_measurement(sentence):
    # Regular expression to match number and unit
    match = re.search(r'(\d+(\.\d+)?)\s*(\w+)', sentence)
    
    if match:
        number = match.group(1)
        unit = match.group(3)
        
        # Convert unit to full form
        full_unit = unit_mapping.get(unit, unit)
        
        # Form the final string
        result = f"{number} {full_unit}"
        return result
    else:
        return None

# Example usage
sentence_1 = "The weight of the ball is 200 g."
sentence_2 = "The width is 5mm."

print(extract_measurement(sentence_1))  # Output: 200 gram
print(extract_measurement(sentence_2))  # Output: 5 millimeter

In [ ]:
for x, y in zip(output, values):
    print("Predicted: " + extract_measurement(x) + " ------" + "Actual: " + y)

In [ ]:
output = forward(ename, img)[0]
print(output)
print("Predicted: " + extract_measurement(output))
print("Actual: " + value)

In [ ]:
cmp = pd.DataFrame({"index": [], "entity_name"=[] ,"prediction": [], "actual": []})

In [ ]:
for index, record in tqdm(subset.iterrows(), total=len(subset)):
    img = url_to_image(record["image_link"])
    ename = record["entity_name"]
    value = record["entity_value"]
    
    pred = extract_measurement(forward(ename, img)[0])
    
    cmp.loc[len(cmp)] = {"entity_name": ename, "prediction": pred, "actual": value}

In [ ]:
cmp

In [ ]:
cmp.loc[len(cmp)] = {"prediction": pred, "actual": value}

In [ ]:
cmp